In [1]:
 # Import modules
import sys
import datetime
import re
from py4j.protocol import Py4JJavaError
from google.cloud import storage

# Import SparkSession
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession \
            .builder \
            .appName("stackoverflow") \
            .getOrCreate()

# Variable section
TABLE = "bigquery-public-data.stackoverflow.posts_questions"
TAG = "asp.net-mvc"

PATH = "tmp" + str(datetime.datetime.now())
BUCKET_NAME = "landing_dataset"
TEMP_GCS_PATH = "gs://" + BUCKET_NAME + "/" + PATH
NEW_PATH = "/".join(["stackoverflow_questions_posts.csv.gz"])
regex_gcs_path = "part-[0-9a-zA-Z\-]*.csv.gz" 

# Create DataFrame from BigQuery table
try:
    df = (spark.read \
              .format("bigquery") \
              .option("table", TABLE) \
              .load())

except Py4JJavaError:
    print(f"{table} does not exist in BigQuery or couldn't be read.")


df_filter = df.where(df.tags == TAG)

# df_filter.coalesce(1).write.csv(TEMP_GCS_PATH)

(df_filter
     .coalesce(1)
     .write
     .options(codec = "org.apache.hadoop.io.compress.GzipCodec")
     .csv(TEMP_GCS_PATH))

# df_filter.show(5)


# create storage client
storage_client = storage.Client()

# create variable representing the temp bucket
source_temp_bucket = storage_client.get_bucket(BUCKET_NAME)


# grab all the files inside bucket (_SUCCESS file + data file)
blobs = list(source_temp_bucket.list_blobs(prefix=PATH))

# for loop to copy the partition data to new location
for blob in blobs:
    if re.search(regex_gcs_path, blob.name):
        source_temp_bucket.copy_blob(blob, source_temp_bucket, NEW_PATH)
        
    
# finally delete temp folder
for blob in blobs:
    blob.delete()        
        

In [5]:
# PATH = "tmp" + str(datetime.datetime.now())
# BUCKET_NAME = " landing_dataset"
# TEMP_GCS_PATH = "gs://" + BUCKET_NAME + "/" + PATH

# print(TEMP_GCS_PATH)

gs:// landing_dataset/tmp2025-10-06 09:43:44.568611
